In [1]:
from statsbombpy import sb
import pandas as pd
from datetime import datetime
from meteostat import Point, Daily
from tqdm import tqdm

In [91]:

stadiums = pd.read_csv("data/stadiums-with-GPS-coordinates.csv")
matches = pd.read_csv("data/premier-league-matches.csv")

# Renaming Columns so it is the same as in the stadiums dataset
matches.rename(columns={"Home":"Team"}, inplace = True)
# Trimming whitespaces in the stadium dataset
stadiums = stadiums.applymap(lambda x: x[:-1] if type(x) == str and x[-1] == " " else x)

# para ser aplicado no stadiums
team_name_map = {
    'Manchester United':'Manchester Utd',
    'Brighton & Hove Albion':'Brighton',
    'Newcastle United': 'Newcastle Utd',
    'West Bromwich Albion' : 'West Brom',
    'Tottenham Hotspur' : 'Tottenham',
    'West Ham United' : 'West Ham',
    'Wolverhampton Wanderers' : 'Wolves'
}

stadiums.Team =  stadiums.Team.replace(team_name_map)

# Contains coordinates and date for each game
display(matches.head())
display(stadiums.head())

matches = matches.merge(stadiums, left_on="Team", right_on='Team')
print(matches.Team.nunique())



,Season_End_Year,Wk,Date,Team,HomeGoals,AwayGoals,Away,FTR
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A


,Team,FDCOUK,City,Stadium,Capacity,Latitude,Longitude,Country
0,Arsenal,Arsenal,London,Emirates Stadium,60361,51.555000,-0.108611,England
1,Aston Villa,Aston Villa,Birmingham,Villa Park,42785,52.509167,-1.884722,England
2,Blackburn Rovers,Blackburn,Blackburn,Ewood Park,31154,53.728611,-2.489167,England
3,Bolton Wanderers,Bolton,Bolton,Reebok Stadium,28100,53.580556,-2.535556,England
4,Chelsea,Chelsea,London,Stamford Bridge,42449,51.481667,-0.191111,England


36


In [46]:

asl = []
for year in range(2018, 2022):
    a = matches[matches.Season_End_Year == year][~matches[matches.Season_End_Year == year].Team.isin(stadiums.Team)].Team.unique()
    asl.append(a.tolist())
    
set(sum(asl,[]))

{'Bournemouth',
 'Brighton',
 'Huddersfield',
 'Manchester Utd',
 'Newcastle Utd',
 'Sheffield Utd',
 'Tottenham',
 'West Brom',
 'West Ham',
 'Wolves'}

In [80]:
stadiums[stadiums.Team.str.match('W')]

,Team,FDCOUK,City,Stadium,Capacity,Latitude,Longitude,Country
17,West Bromwich Albion,West Brom,West Bromwich,The Hawthorns,27877,52.509167,-1.963889,England
18,Wigan Athletic,Wigan,Wigan,DW Stadium,25133,53.547778,-2.653889,England
19,Wolverhampton Wanderers,Wolves,Wolverhampton,Molineux Stadium,27828,52.590278,-2.130278,England
42,Watford,Watford,Watford,Vicarage Road,23500,51.649836,-0.401486,England
43,West Ham United,West Ham,London,Boleyn Ground,35303,51.531944,0.039444,England
78,Werder Bremen,Werder Bremen,Bremen,Weserstadion,42100,53.066394,8.837628,Germany


In [61]:
stadiums[stadiums.Country=='England'].Team.tolist()

['Arsenal',
 'Aston Villa',
 'Blackburn Rovers',
 'Bolton Wanderers',
 'Chelsea',
 'Everton',
 'Fulham',
 'Liverpool',
 'Manchester City',
 'Manchester United',
 'Newcastle United',
 'Norwich City',
 'Queens Park Rangers',
 'Stoke City',
 'Sunderland',
 'Swansea City',
 'Tottenham Hotspur',
 'West Bromwich Albion',
 'Wigan Athletic',
 'Wolverhampton Wanderers',
 'Barnsley',
 'Birmingham City',
 'Blackpool',
 'Brighton & Hove Albion',
 'Bristol City',
 'Burnley',
 'Cardiff City',
 'Coventry City',
 'Crystal Palace',
 'Derby County',
 'Doncaster Rovers',
 'Hull City',
 'Ipswich Town',
 'Leeds United',
 'Leicester City',
 'Middlesbrough',
 'Millwall',
 'Nottingham Forest',
 'Peterborough United',
 'Portsmouth',
 'Reading',
 'Southampton',
 'Watford',
 'West Ham United']

In [92]:
# Now we're gonna use the meteostat library to retrieve weather data for each match

output_file = "data/weather.csv"
columns = [
    'stadium', 
    'date', 
    'tavg', 
    'tmin', 
    'tmax', 
    'prcp', 
    'snow', 
    'wdir', 
    'wspd', 
    'wpgt', 
    'pres',
    'tsun']

# Creating header
with open(output_file, 'w') as f:
    f.write(columns[0])
    for col in columns[1:]:
        f.write(",")
        f.write(col)
    f.write("\n")
    
for i in tqdm(range(len(matches))):
    stadium = matches["Stadium"][i]
    latitude = float(matches["Latitude"][i])
    longitude = float(matches["Longitude"][i])
    date_str = matches["Date"][i]
    year, month, day = map(int, date_str.split("-"))
    
    # API Call
    start = datetime(year, month, day)
    end   = datetime(year, month, day)
    location = Point(latitude, longitude, 70)
    data = Daily(location, start, end)
    data = data.fetch()
    
    # Data not found
    if data.empty:
        continue
    # Writing data
    with open(output_file, 'a') as f:
        f.write(stadium)
        f.write(",")
        f.write(date_str)
        for col in columns[2:]:
            f.write(",")
            f.write(str(data[col][0]))
        f.write("\n")

  0%|          | 0/10316 [00:00<?, ?it/s]

  7%|▋         | 744/10316 [00:38<06:15, 25.46it/s]  

In [ ]:
weather_df = pd.read_csv("data/weather.csv")
matches_with_weather = matches.merge(weather_df, left_on = ["Stadium","Date"], right_on = ["stadium","date"], copy=True)

In [ ]:
matches_with_weather.to_pickle('bkp_matches_w_weather.pkl')